In [ ]:
# Reads in photometry from different sources, normalizes them, and puts them
# onto a BJD time scale

# Created 2021 Dec. 28 by E.S.

In [1]:
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

%matplotlib qt

In [2]:
## USER INPUTS

photometry_source_file = "t_sex_asas3_vband_prepolished.txt"
epoch_writeout_file = "junk_hjds_t_sex.csv"
bjd_file = "t_sex_asas3_vband_bjd.csv" # generated as-we-go
writeout_polished_file = "t_sex_asas3_vband_polished.txt"

In [3]:
stem_super = "/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/"
stem_moore_macadam = stem_super + "polished_macadam_moore_photometry/"
stem_public_sources = stem_super + "all_photometry_program_stars/polished/"
bjd_sources = stem_super + "all_photometry_program_stars/bjds/"

In [4]:
# read in photometry

#df = pd.read_csv(stem_public_sources + photometry_source_file)
df = pd.read_csv(stem_public_sources + photometry_source_file, delim_whitespace=True)

In [5]:
# if there's a bunch of too-old stuff, remove it
#df = df.where(np.logical_and(df["JD"]>2456474-400,df["JD"]<2456474+400)).dropna(how="all").reset_index()
#df = df.where(df["JD"]<=2459238.567486600).dropna(how="all").reset_index()

In [6]:
df.keys()

Index(['HJD', 'MAG_2', 'MAG_0', 'MAG_1', 'MAG_3', 'MAG_4', 'MER_2', 'MER_0',
       'MER_1', 'MER_3', 'MER_4', 'GRADE', 'FRAME'],
      dtype='object')

In [7]:
df["HJD"] = np.add(2450000,df["HJD"])

In [8]:
# write out only JD, for conversion

#df["JD"] = np.add(df["obsmjd"],2400000.5)
#df["JD"].to_csv("junk_jds.csv", header=False, index=False)
df["HJD"].to_csv(epoch_writeout_file, header=False, index=False)

# at this point, do conversion step manually online

In [9]:
# FYI: location of Wichita (approx. center of U.S.)

'''
lat_wichita = 37.688889
lon_wichita = -97.336111
'''

'\nlat_wichita = 37.688889\nlon_wichita = -97.336111\n'

# read bjds back in and tack onto the dataframe

In [8]:
intermed_bjd = pd.read_csv(bjd_sources + bjd_file, names=["BJD"])
print(bjd_sources + bjd_file)

df["BJD"] = intermed_bjd.values

#df = df.join(intermed_bjd["BJD"])

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/bjds/t_sex_asas3_vband_bjd.csv


In [9]:
len(df)

504

In [10]:
df

,HJD,MAG_2,MAG_0,MAG_1,MAG_3,MAG_4,MER_2,MER_0,MER_1,MER_3,MER_4,GRADE,FRAME,BJD
0,2.452054e+06,9.876,9.833,9.863,9.868,9.872,0.036,0.049,0.046,0.032,0.032,A,19978,2.452054e+06
1,2.452069e+06,9.815,9.732,9.805,9.813,9.815,0.024,0.045,0.028,0.028,0.032,A,22224,2.452069e+06
2,2.452442e+06,10.167,10.147,10.170,10.173,10.186,0.064,0.071,0.068,0.069,0.076,D,8406,2.452442e+06
3,2.452661e+06,9.868,9.824,9.863,9.865,9.869,0.037,0.048,0.039,0.042,0.044,A,35993,2.452661e+06
4,2.452713e+06,10.098,10.090,10.079,10.098,10.094,0.027,0.036,0.032,0.028,0.030,A,45107,2.452713e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2.453386e+06,9.871,9.926,9.881,9.877,9.865,0.032,0.060,0.033,0.030,0.036,A,126078,2.453386e+06
500,2.453391e+06,10.076,10.061,10.066,10.071,10.068,0.027,0.044,0.026,0.025,0.030,A,126762,2.453391e+06
501,2.453394e+06,10.210,10.226,10.225,10.217,10.203,0.048,0.062,0.040,0.041,0.055,B,127050,2.453394e+06
502,2.453400e+06,9.836,9.832,9.825,9.832,9.827,0.037,0.052,0.038,0.035,0.040,A,127807,2.453400e+06


In [19]:
# in case of weird naming conventions
'''
df["Magnitude"] = df["MAG_0"]
df["Uncertainty"] = df["MER_0"]
'''

# look at the data, choose what we want

In [13]:
plt.clf()
plt.scatter(df["BJD"],
            df["Magnitude"])
plt.show()

KeyError: 'Magnitude'

<Figure size 432x288 with 0 Axes>

In [11]:
plt.clf()
plt.scatter(df["BJD"],
            df["MAG_0"])
plt.scatter(df["BJD"],
            df["MAG_1"])
plt.scatter(df["BJD"],
            df["MAG_2"])
plt.scatter(df["BJD"],
            df["MAG_3"])
plt.scatter(df["BJD"],
            df["MAG_4"])
plt.show()

In [14]:
plt.clf()
plt.scatter(df["BJD"],
            df["mag_autocorr"])
plt.show()

In [13]:
plt.clf()
plt.scatter(df["JD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="k")
plt.scatter(df["BJD"].where(df["Band"] == "V"),
            df["Magnitude"].where(df["Band"] == "V"), color="b")
plt.show()

In [18]:
plt.clf()
plt.scatter(df["BJD"].where(df["Observer Code"] == "SAH"),
            df["Magnitude"].where(df["Observer Code"] == "SAH"), color="k")
plt.show()

In [13]:
df.keys()

Index(['JD', 'Magnitude', 'Uncertainty', 'HQuncertainty', 'Band',
       'Observer Code', 'Comment Code(s)', 'Comp Star 1', 'Comp Star 2',
       'Charts', 'Comments', 'Transfomed', 'Airmass', 'Validation Flag',
       'Cmag', 'Kmag', 'HJD', 'Star Name', 'Observer Affiliation',
       'Measurement Method', 'Grouping Method', 'ADS Reference', 'Digitizer',
       'Credit', 'BJD'],
      dtype='object')

In [12]:
print(np.min(df["JD"]))
print(np.max(df["JD"]))

2439144.571
2459546.39706


In [11]:
len(df)

30529

In [15]:
df["Band"].value_counts()

KeyError: 'Band'

In [16]:
df["Observer Code"].value_counts()

KeyError: 'Observer Code'

In [12]:
# choose photometric subset

#df_subset_photo = df
#df_subset_photo = df.where(np.logical_and(df["BJD"] > 2452456,df["Band"] == "V"))
#df_subset_photo = df.where(np.logical_or(df["BJD"] < 2455109,df["BJD"] > 2455535))
#df_subset_photo = df.where(df["Band"] == "V")
df_subset_photo = df.where(df["MAG_0"] < 50)
#df_subset = df[["JD", "sdf"]]

In [14]:
# check to confirm
plt.clf()
#plt.scatter(df_subset_photo["BJD"],df_subset_photo["Magnitude"])
plt.scatter(df_subset_photo["BJD"],df_subset_photo["MAG_0"])
plt.show()

In [13]:
np.max(df_subset_photo["BJD"])

2456809.3022545683

In [14]:
df_subset_photo.keys()

Index(['index', 'JD', 'Magnitude', 'Uncertainty', 'HQuncertainty', 'Band',
       'Observer Code', 'Comment Code(s)', 'Comp Star 1', 'Comp Star 2',
       'Charts', 'Comments', 'Transfomed', 'Airmass', 'Validation Flag',
       'Cmag', 'Kmag', 'HJD', 'Star Name', 'Observer Affiliation',
       'Measurement Method', 'Grouping Method', 'ADS Reference', 'Digitizer',
       'Credit', 'BJD'],
      dtype='object')

# feature scale the photometry

### wait! scaling of error bars won't be the same, since an offset is being subtracted off

In [29]:
'''
scaler = MinMaxScaler() # min-max mapped to 0-to-1
df_subset_photo[['mag_scaled']] = scaler.fit_transform(df_subset_photo[['Magnitude']])
# df_subset_photo[['error_scaled']] = scaler.transform(df_subset_photo[['Uncertainty']])
'''

In [21]:
'''
plt.scatter(df_subset_photo["BJD"],df_subset_photo["mag_scaled"])
plt.show()
'''

# write out as 'polished' photometry

In [ ]:
# cols BJD, mag, error

In [15]:
# remove rows of all NaNs

df_subset_photo_dropna = df_subset_photo.dropna(axis=0, how='all')

In [17]:
# write out only the relevant cols

df_subset_photo_dropna.to_csv(stem_public_sources + writeout_polished_file, 
                              columns=["BJD","Magnitude","Uncertainty"], index=False)
print(stem_public_sources + writeout_polished_file)

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['Magnitude', 'Uncertainty'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [18]:
# write out only the relevant cols

# naming convention change
df_subset_photo_dropna["Magnitude"] = df_subset_photo_dropna["MAG_0"]
df_subset_photo_dropna["Uncertainty"] = df_subset_photo_dropna["MER_0"]

df_subset_photo_dropna.to_csv(stem_public_sources + writeout_polished_file, 
                              columns=["BJD","Magnitude","Uncertainty"], index=False)
print(stem_public_sources + writeout_polished_file)

/Users/bandari/Documents/git.repos/rrlyrae_metallicity/notebooks_for_development/all_photometry_program_stars/polished/t_sex_asas3_vband_polished.txt


/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/bandari/anaconda3/envs/rrl_env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
writeout_polished_file

't_sex_asas3_vband_polished.txt'